In this work, I found out that the text was stored in binary tree after decomposing based on sentences. All the nodes and leaf are stored in the same column. And some trees are incomplete with leaf missing. The number of words closely linked with the level of the nodes. So in this work, I grouped the sentences based on their word count and then conduct the analysis.

experiment One, all the data are used for the training.
experiment Two, I grouped the sentences based on their word count and trained them separately.
experiment Three, a threshold is introduced. The data below the threshold are trained separately, while those above the threshold are trained in a group.
The accuracy in experiment One is 0.32464436755094195. In experiment Two, I found out that the accuracy of sentences with more than 7 words (including 7 words) is lower than that in experiment One. Therefore, I choose 6 as the threshold in experiment Three. Which means, the sentences with less than 7 words are trained separately and those with more than 6 words are trained in a group.

In [ ]:
import numpy as np
import pandas as pd
from textblob import TextBlob
#from textblob.classifiers import NaiveBayesClassifier
from textblob.sentiments import NaiveBayesAnalyzer, PatternAnalyzer
#from textblob import Blobber

from nltk.classify import NaiveBayesClassifier
from nltk.corpus import stopwords

from sklearn.metrics import accuracy_score

import nltk

import matplotlib.pyplot as plt

import random 
import wordcloud

In [ ]:
df = pd.read_csv(filepath_or_buffer='../input/train.tsv', sep='\t', index_col=0)
df.shape

In [ ]:
df.head()

In [ ]:
test_df = pd.read_csv(filepath_or_buffer='../input/test.tsv', sep='\t', index_col=0)
test_df.shape

In [ ]:
test_df.head()

# EDA

In [ ]:
eda_df = df.copy()
eda_df = eda_df.reset_index()
eda_df.head()

In [ ]:
eda_df.groupby(['Sentiment']).count().T.plot()

From above plot, we can see sentiment 2 is much higher than rest of others. sentiment 1 & 3 are close and 0 & 4 are close. In this data, sentiment 0 and 4 are quite few.

In [ ]:
eda_df['len'] = eda_df['Phrase'].apply(lambda x: len(TextBlob(x).words))

In [ ]:
eda_df.groupby(['len', 'Sentiment'])['Phrase'].count().unstack().plot(kind='bar', stacked=False)

In the above plot, we can see that the sentiment is related to the sentences word count. And sentiment 1 & 2 mainly turn up when the sentence word count is between 1 and 6.

In [ ]:
eda_df.loc[eda_df['Sentiment'] == 0].groupby(['len', 'Sentiment'])['Phrase'].count().unstack().plot(kind='bar', stacked=False)
eda_df.loc[eda_df['Sentiment'] == 1].groupby(['len', 'Sentiment'])['Phrase'].count().unstack().plot(kind='bar', stacked=False)
eda_df.loc[eda_df['Sentiment'] == 2].groupby(['len', 'Sentiment'])['Phrase'].count().unstack().plot(kind='bar', stacked=False)
eda_df.loc[eda_df['Sentiment'] == 3].groupby(['len', 'Sentiment'])['Phrase'].count().unstack().plot(kind='bar', stacked=False)
eda_df.loc[eda_df['Sentiment'] == 4].groupby(['len', 'Sentiment'])['Phrase'].count().unstack().plot(kind='bar', stacked=False)

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

from nltk import FreqDist
from nltk.stem import SnowballStemmer,WordNetLemmatizer
stemmer=SnowballStemmer('english')
lemma=WordNetLemmatizer()
from string import punctuation
import re


In [ ]:
def clean_review(review):
    review=[lemma.lemmatize(w) for w in word_tokenize(str(review).lower())]
    review=' '.join(review)
    return review

In [ ]:
eda_df['cleaned_pharse']=eda_df.Phrase.apply(lambda x: clean_review(x))
eda_df.head()

In [ ]:
from wordcloud import WordCloud, STOPWORDS
stopwords = set(STOPWORDS)

In [ ]:
def grey_color_func(word, font_size, position, orientation, random_state=None, **kwargs):
    return "hsl(0, 0%%, %d%%)" % random.randint(1, 20)

In [ ]:
def showWordCloud(text,title):
    stp = STOPWORDS.copy()
    #STOPWORDS.add( "code")
    wc = WordCloud(background_color="white", max_words=50,  stopwords=stp)
    # generate word cloud
    wc.generate(text)

    # show
    plt.figure(figsize=(10,20))
    plt.imshow(wc.recolor(color_func=grey_color_func, random_state=3))
    plt.axis("off")
    plt.title(title)
    plt.show()

In [ ]:
showWordCloud(str(eda_df['cleaned_pharse'].tolist()), 'top 50 word cloud for all Sentiment')

In [ ]:
showWordCloud(str(eda_df.loc[eda_df['Sentiment'] == 0]['cleaned_pharse'].tolist()), 'top 50 word cloud for all Sentiment 0')

In [ ]:
showWordCloud(str(eda_df.loc[eda_df['Sentiment'] == 4]['cleaned_pharse'].tolist()), 'top 50 word cloud for all Sentiment 4')

# enrich features

In [ ]:
df['len'] = df['Phrase'].apply(lambda x: len(TextBlob(x).words))

In [ ]:
df['cleaned_pharse']=df.Phrase.apply(lambda x: clean_review(x))
df.head()

In [ ]:
def prepare_train_data(data):
    data['len'] = data['cleaned_pharse'].apply(lambda x: len(TextBlob(x).words))
    data['word_feats'] = data.apply(lambda x: word_feats(x['cleaned_pharse']), axis=1)
    data['Sent_str'] = data['Sentiment'].apply(str)
    return data

In [ ]:
def prepare_test_data(data):
    data['len'] = data['cleaned_pharse'].apply(lambda x: len(TextBlob(x).words))
    data['word_feats'] = data.apply(lambda x: word_feats(x['cleaned_pharse']), axis=1)
    return data

In [ ]:
def get_train_X(data):
    data = data.apply(lambda x: (x['word_feats'],x['Sentiment']), axis = 1)
    return list(data)

# Experiment

In [ ]:
solution_accuracy = []

In [ ]:
#stopset = list(set(stopwords.words('english')))

def word_feats(words):
    return dict([(word, True) for word in words.split()])# if word not in stopset])

## try all data

In [ ]:
train_df = df.copy()

In [ ]:
train_df['word_feats'] = train_df.apply(lambda x: word_feats(x['cleaned_pharse']), axis=1)
train_df.head()

In [ ]:
train_df['Sent_str'] = train_df['Sentiment'].apply(str)
train_df.head()

In [ ]:
features = list(train_df.apply(lambda x: (x['word_feats'],x['Sentiment']), axis = 1))

In [ ]:
cutoff = (int)(len(features)*3/4)
print(cutoff)
trainfeats = features[:cutoff]
testfeats = features[cutoff:] 
print(len(trainfeats))
print(len(testfeats))

In [ ]:
import nltk

nltk_class = nltk.classify.NaiveBayesClassifier
#cl = nltk_class.train(testfeats)
#print('train on %d instances, test on %d instances' % (len(trainfeats), len(testfeats)))
 
classifier = nltk_class.train(trainfeats)
acc = nltk.classify.util.accuracy(classifier, testfeats)
print( 'accuracy for all data:', acc)
#classifier.show_most_informative_features()


## use multi classfiy based on length of word

In [ ]:
def init_experiment_features(experiment_name, experiment_data):
#     print(experiment_name + ' Start:')
    features = list(experiment_data.apply(lambda x: (x['word_feats'],x['Sentiment']), axis = 1))
    cutoff = (int)(len(features)*3/4)
#     print(cutoff)
    trainfeats = features[:cutoff]
    testfeats = features[cutoff:] 
#     print(len(trainfeats))
#     print(len(testfeats))
    #return trainfeats, testfeats
    nltk_class = nltk.classify.NaiveBayesClassifier
#     print('train on %d instances, test on %d instances' % (len(trainfeats), len(testfeats)))

    classifier = nltk_class.train(trainfeats)
    acc = nltk.classify.util.accuracy(classifier, testfeats)
#     print( 'accuracy:', acc)
    #classifier.show_most_informative_features()
#     print(experiment_name + ' Done.')
    return acc

In [ ]:
acc1 = init_experiment_features('Is 1 ', train_df.loc[train_df['len'] == 1])
acc2 = init_experiment_features('Bigger than 1 ', train_df.loc[train_df['len'] > 1])
print('acc for just Sentiment == 1:' + str(acc1))
print('acc for just Sentiment > 1:' + str(acc2))

## Try all len

In [ ]:
acc_list = []
len_count = len(train_df.groupby('len').count().reset_index()) #.uniques()
# acc1 = init_experiment_features('Is 1 ', train_df.loc[train_df['len'] == 1])
# acc2 = init_experiment_features('Bigger than 1 ', train_df.loc[train_df['len'] > 1])

for l in range(1, len_count):
    acc_list.append( init_experiment_features(str(l), train_df.loc[train_df['len'] == l]))

acc_pd = pd.Series(acc_list).reset_index().T

In [ ]:
tt = pd.DataFrame(acc_list, columns=['acc'])#.rename('acc')
tt.T

In [ ]:
tt.plot()

choose all from length 5, because from length 5 acc is low than 1 vs all.

In [ ]:
acc_list = []

len_knot = 7

for l in range(1, len_knot):
    acc_list.append( init_experiment_features(str(l), train_df.loc[train_df['len'] == l]))

acc_list.append( init_experiment_features(str(l), train_df.loc[train_df['len'] > len_knot]))

In [ ]:
acc_list

In [ ]:
tt = pd.DataFrame(acc_list, columns=['acc'])
print(tt.T)
print()
print('average score : ' + str((tt.sum() / len(tt)).values[0]))
tt.plot()


## Conclusion
- use all data 0.3230552351659618
- use len as Fields, 1 vs others. 
    - 1: 0.7326381184823608
    - others: 0.3166483102428845
- each len as a group, 
    - 	0	1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	20	21	22	23	24	25	26	27	28	29	30	31	32	33	34	35	36	37	38	39	40	41	42	43	44	45	46	47
    - acc	0.732638	0.63291	0.568009	0.502189	0.462659	0.373866	0.341786	0.323549	0.319231	0.279503	0.26087	0.235838	0.260292	0.291045	0.254125	0.210037	0.25641	0.220379	0.221607	0.313253	0.273026	0.265152	0.2	0.21164	0.226994	0.236486	0.315789	0.237113	0.336957	0.22973	0.397059	0.2	0.230769	0.121212	0.142857	0.090909	0.105263	0.133333	0.333333	0.25	0.222222	0.0	0.333333	0.0	0.0	0.0	0.5	0.0
- chose 7 as knot, 1,2,3,4,5,6,7 train a classify, others train one.
    -             0         1         2        3         4         5         6         7         8
    - 0.7326381184823608, 0.632909520993495, 0.5680089070328447, 0.5021893814997264, 0.46265938069216755, 0.3738656987295826, 0.34178610804851156, 0.31660417267570423
    
** The conculsion is chose 7 as knot is a good choose. **

# Run Real Data

In [ ]:
def get_test_X(data):
    return data['word_feats']

In [ ]:
train_data = prepare_train_data(df)

In [ ]:
test_df['cleaned_pharse']=test_df.Phrase.apply(lambda x: clean_review(x))
test_df.head()

In [ ]:
test_data = prepare_test_data(test_df)

In [ ]:
def do_train_classify(train_X, test_X):
    nltk_class = nltk.classify.NaiveBayesClassifier
    classifier = nltk_class.train(train_X)    
    
    for i, row in test_X.iteritems():
        t_score = classifier.classify(test_X.loc[i])
        #print('      >>> processing item: ' + str(i) + " >>> score: " + str(t_score))
        res_df.loc[i,'Sentiment'] = t_score

In [ ]:
test_df.head()

In [ ]:
res_df = test_df.copy()
#res_df = res_df.reset_index()
res_df['Sentiment'] = 2
#res_df = res_df[['PhraseId','Sentiment','len']]
res_df.head()

In [ ]:
len_knot = 7

for l in range(1, len_knot):
    print('turn : ' + str(l))
    train_X = get_train_X(train_data.loc[train_data['len'] == l])
    test_X = get_test_X(test_data.loc[test_data['len'] == l])
    do_train_classify(train_X, test_X)

print('The last turn!')
train_X = get_train_X(train_data.loc[train_data['len'] > len_knot])
test_X = get_test_X(test_data.loc[test_data['len'] > len_knot])
do_train_classify(train_X, test_X)

In [ ]:
res_df.head()

In [ ]:
len(test_df)

In [ ]:
len(res_df.loc[res_df['Sentiment'] != -1])

In [ ]:
len(res_df.loc[res_df['Sentiment'] == -1])

In [ ]:
len(res_df)

In [ ]:
submission = res_df.reset_index()
submission = submission[['PhraseId','Sentiment']]
submission.head()

In [ ]:
len(test_df)

In [ ]:
len(submission)

In [ ]:
submission.to_csv('submission.csv', index=False)